## Linear Regression: Best Subset and Stepwise Regression

**Functions**

`np.linalg.lstsq`, `sklearn.model_selection.cross_val_score`, `sklearn.linear_model.LinearRegression`, `sklearn.model_selection.KFold`

In [1]:
import numpy as np
import pandas as pd

### Exercise 41
Download data from [Ken French's website](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/) on the VWM and the 12 industry portfolios. Reformat both data sets so that they has a `DatetimeIndex`.


In [2]:
vwm = pd.read_csv("data/VWM.csv", index_col="Date")
vwm.index = pd.to_datetime(vwm.index, format="%Y%m")
vwm = vwm.resample("M").last()

industries = pd.read_csv("data/12_Industry_portfolios.csv", index_col="Date")
industries.index = pd.to_datetime(industries.index, format="%Y%m")
industries = industries.resample("M").last()

### Exercise 42

Use Best Subset Regression with cross-validation to select the weights of a tracking portfolio where the industry portfolios are used to track the value-weighted-market. Use data until the end of 2014. A tracking portfolio is a portfolio that replicates a portfolio using other assets. The weights can be estimated using a regression model excludes a constant.

$$ R_{i,p} = \beta_1 R_{i,1} + \beta_2 R_{i,2} + \ldots + \beta_k R_{i,k} + \epsilon_{i} $$

where $R_{i,j}$ are returns on the assets used to construct the tracking portfolio. The constant is excluded since the portfolio should track both the shock and match the mean return. OLS minimizes the variance of the tracking error (in-sample).

In [3]:
x = industries["1980":"2014"]
y = vwm["VWM"]["1980":"2014"]
t, p = x.shape

#### Explanation

We start by downloading the data which have all been downloaded from [Ken French's website](https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html).  In this exercise we will use the Value-weighted Market Return and 12 industry portfolios.

The data is subsetted into 1980 - 2014 (inclusive) and the data from 2015 to the end of the sample will be used to evaluate model performance.

In [4]:
from itertools import combinations

for i in range(1, p + 1):
    count = 0
    for comb in combinations(x.columns, i):
        print(comb)
        count += 1
        if count > 1:
            break

('NoDur',)
('Durbl',)
('NoDur', 'Durbl')
('NoDur', 'Manuf')
('NoDur', 'Durbl', 'Manuf')
('NoDur', 'Durbl', 'Enrgy')
('NoDur', 'Durbl', 'Manuf', 'Enrgy')
('NoDur', 'Durbl', 'Manuf', 'Chems')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'BusEq')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'Telcm')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 'Telcm')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 'Utils')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 'Telcm', 'Utils')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 'Telcm', 'Shops')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 'Telcm', 'Utils', 'Shops')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 'Telcm', 'Utils', 'Hlth ')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 'Telcm', 'Utils', 'Shops', 'Hlth ')
('NoDur', 'Durbl', 'Manuf', 'Enrgy', 'Chems', 'BusEq', 'Telcm', 'Utils', 'Shops

#### Explanation

We can use `combinations` from `itertools` to construct all combinations of `k` elemments from a list with `p`.  Looping from `k=1` to `p` allows all distinct combinations to be constructed.

In [5]:
from numpy.linalg import lstsq

best_models = {}
for i in range(1, p + 1):
    best_sse = np.inf
    count = 0
    for comb in combinations(x.columns, i):
        if count > 1:
            break
        reg = x[list(comb)]
        beta = lstsq(reg, y, rcond=None)[0]
        resid = y - reg @ beta
        sse = resid @ resid
        if sse < best_sse:
            best_sse = sse
            best_models[i] = list(comb)
pd.Series(best_models)

1                                               [Other]
2                                        [BusEq, Money]
3                                 [NoDur, BusEq, Other]
4                          [NoDur, Enrgy, BusEq, Money]
5                   [NoDur, Enrgy, BusEq, Money, Other]
6            [NoDur, Enrgy, BusEq, Telcm, Money, Other]
7     [NoDur, Enrgy, BusEq, Telcm, Hlth , Money, Other]
8     [NoDur, Manuf, Enrgy, BusEq, Telcm, Hlth , Mon...
9     [NoDur, Manuf, Enrgy, BusEq, Telcm, Shops, Hlt...
10    [NoDur, Manuf, Enrgy, BusEq, Telcm, Utils, Sho...
11    [NoDur, Manuf, Enrgy, Chems, BusEq, Telcm, Uti...
12    [NoDur, Durbl, Manuf, Enrgy, Chems, BusEq, Tel...
dtype: object

#### Explanation

Best subset regression involves fitting all possible model.  Python's `itertools` module has a function `combinations` that will construct all distinct combinations of $n$ items from a collection. This can be used with different $n$ to build all possible models. Below we see that it outputs list of columns names.

We use `lstsq` from `numpy.linalg` to estimate the regression parameters.  This function is quite a bit faster than `OLS` from `statsmodels. It returns multiple outputs but we only need the first which is selected using `[0]`.

The best model is selected by iterating across all model sizes and retaining the model with the smallest SSE. The variable `best_sse` is initialized to be infinity so that at least one model will always be smaller.

Finally, we store the columns of the best model in a dictionary where the key is the number of variables in the model. The last line prints the set of models selected.

**Caution**

The number of distinct models grows at rate $2^p$ and so this code may be very slow when $p$ is modestly large.

In [6]:
def xval_5fold(y, x, random=False, seed=20201231):
    # Use numpy arrays for simplicity
    y = np.asarray(y)
    x = np.asarray(x)
    n = y.shape[0]
    if random:
        # If randomization is needed, use the seed
        rg = np.random.default_rng(seed)
        # Generate a set of index values to use to randomly reorder the data
        # After randomization, we can use the data as if it is inorder
        ind = rg.permutation(np.arange(n))
        y = y[ind]
        x = x[ind]
    # Compute the block size
    block = n / 5.0
    sse = 0.0
    for i in range(5):
        # Start and End of each block need to be integers
        # Rounding ensures that we get all observations since int rounds down
        st = int(np.round(i * block))
        en = int(np.round((i + 1) * block))
        # Constrct the indicaes of the observations that we leave out
        leave_out = np.r_[st:en]
        # The included are those that we don't leave out
        include = np.setdiff1d(np.arange(n), leave_out)
        # Compute the regression coefficients
        beta = lstsq(x[include], y[include], rcond=None)[0]
        # Compute the residuals and sdd to the SSE
        resid = y[st:en] - x[st:en] @ beta
        sse += resid @ resid
    return sse


xval_5fold(y, x[best_models[2]])

771.9822535009072

#### Explanation

We need to cross-validate the set of best models. To do this, we can write a function (see the method available through scikit-learn that will simplify this step) so that we can reuse it later. The function defaults to non-random cross-validation which will leave out blocks containing 20% of the data in order. The function can optionally be used to randomly select the data left out.

When we randomize we get different values.  Randomization may select different models.  It usually makes little difference.

In [7]:
xval_5fold(y, x[best_models[2]], random=True)

750.3129727221956

#### Explanation 

We can apply `xval_5fold` to the data to compute the cross-validate SSE, and retain the best. We loop over the keys in the `best_models` and select the contents of the dictionary using `best_models[n_var]`.

While some of the cross-validated SSEs are very large, many are similar to the best. 

In [8]:
bsr_sse_xv = {}
for n_var in best_models:
    bsr_sse_xv[n_var] = xval_5fold(y, x[best_models[n_var]])
bsr_sse_xv = pd.Series(bsr_sse_xv)

print(f"The minimum k is {bsr_sse_xv.idxmin()}")
print(f"The selected model uses {best_models[bsr_sse_xv.idxmin()]}")
bsr_model = best_models[bsr_sse_xv.idxmin()]
bsr_sse_xv

The minimum k is 9
The selected model uses ['NoDur', 'Manuf', 'Enrgy', 'BusEq', 'Telcm', 'Shops', 'Hlth ', 'Money', 'Other']


1     1148.876144
2      771.982254
3      494.175842
4      368.090926
5      283.350797
6      235.289796
7      210.449663
8      208.251414
9      204.369481
10     208.891542
11     209.923549
12     209.984916
dtype: float64

#### Explanation

If we use randomization we may select a different model. The alternative model will usually have an SSE that is close to the SSE of the previous model selected.

In [9]:
sse_xv = {}
for k in best_models:
    sse_xv[k] = xval_5fold(y, x[best_models[k]], random=True)
sse_xv = pd.Series(sse_xv)
print(f"The minimum k is {sse_xv.idxmin()}")
print(f"The selected model uses {best_models[sse_xv.idxmin()]}")
sse_xv

The minimum k is 10
The selected model uses ['NoDur', 'Manuf', 'Enrgy', 'BusEq', 'Telcm', 'Utils', 'Shops', 'Hlth ', 'Money', 'Other']


1     1142.713474
2      750.312973
3      469.750212
4      334.086464
5      246.304735
6      197.971097
7      175.919440
8      161.668160
9      157.701899
10     156.962597
11     157.765810
12     158.046894
dtype: float64

### Exercise 43

Select the best tracking portfolio using Forward Stepwise Regression.

#### Explanation

Forward Stepwise Regression is implemented by tracking the variables that have been included in the model. We then add one variables from those that are excluded to the included and find the model that minimizes the SSE from the next iteration. The is repeated until all variables are added.  The list of included contains the variables in the order they are added, so that `included[:k]` returns the list of included variables in a model containing `k` variables.

In [10]:
# Initialize the list of included which will store variables in order
included = []

for i in range(p):
    # Excluded are the columns that are not in included
    excluded = [col for col in x if col not in included]
    # Initialize to inf so that it will be beated by some model
    best_sse = np.inf
    # Loop across the excludded
    for col in excluded:
        # Add the new column to the included list and select the x data
        try_x = x[included + [col]]
        # Compute the regression and the SSE
        beta = lstsq(try_x, y, rcond=None)[0]
        resid = y - try_x @ beta
        sse = resid @ resid
        # If this sse is better than the best seen, keep it and retain the new variable added
        if sse < best_sse:
            best_sse = sse
            next_var = col
    # Add the variable that reduced the SSE the most
    included.append(next_var)

print(included)

['Other', 'BusEq', 'NoDur', 'Money', 'Enrgy', 'Telcm', 'Hlth ', 'Manuf', 'Shops', 'Utils', 'Chems', 'Durbl']


#### Explanation 

Next we cross-validate the FSR sequence of models by looping across the list of included variables, in order.

We store the cross-validated SSE as a dictionary that we can convert to a `Series`. We can then get the best using `idxmin()` to return the index associated with the smallest cross-validated SSE.

In [11]:
fsr_sse_sv = {}
for i in range(1, p + 1):
    fsr_sse_sv[i] = xval_5fold(y, x[included[:i]])
fsr_sse_sv = pd.Series(fsr_sse_sv)
print(f"The minimum k is {fsr_sse_sv.idxmin()}")
forstep_model = included[:fsr_sse_sv.idxmin()]
print(f"The selected variables are {forstep_model}")
fsr_sse_sv

The minimum k is 9
The selected variables are ['Other', 'BusEq', 'NoDur', 'Money', 'Enrgy', 'Telcm', 'Hlth ', 'Manuf', 'Shops']


1     1148.876144
2      785.510416
3      494.175842
4      373.038967
5      283.350797
6      235.289796
7      210.449663
8      208.251414
9      204.369481
10     208.891542
11     209.923549
12     209.984916
dtype: float64

### Exercise 44

Use Backward Stepwise Regression to select the tracking portfolio.

#### Explanation

Backward Stepwise Regression is substantially similar to Forward except that we being with the included set contains all columns and then remove one at a time.  We always retain the model with the smallest SSE that has one variable removed from the previous model.

We can then cross-validate the BSR models using the same code as for FSR

In [12]:
# Initialize the set of included to all columns
included = list(x.columns)
# Initialize the list of removed variables
removed = []

for i in range(p):
    # Ensure the at least one model will beat the best SSE
    best_sse = np.inf
    # Iteratove over columns that are in the model
    for col in included:
        # Copy the included list
        try_col = included[:]
        # and remove col from the list
        try_col.remove(col)
        # Get the set of x data for these columns
        try_x = x[try_col]
        # Compute the coefficients and sse
        beta = lstsq(try_x, y, rcond=None)[0]
        resid = y - try_x @ beta
        sse = resid @ resid
        # If the sse is less than the best seen so far, retain it
        if sse < best_sse:
            # Retain the sse and update the next to drop variable
            best_sse = sse
            next_drop = col
    # Add the next variable to drop
    removed.append(next_drop)
    # Remove the variable that we drop so that it doesn't show up in the next round
    included.remove(next_drop)

# The list of variables dropped, in order
print(removed)

['Durbl', 'Chems', 'Utils', 'Shops', 'Manuf', 'Hlth ', 'Telcm', 'Other', 'Enrgy', 'NoDur', 'BusEq', 'Money']


In [13]:
# Reversing the dropped list gets us the list of variables included fo
# any number of regressors
included = removed[::-1]

# This is identical to the FSR cross-validation code
bsr_sse_sv = {}
for i in range(1, p + 1):
    bsr_sse_sv[i] = xval_5fold(y, x[included[:i]])
bsr_sse_sv = pd.Series(bsr_sse_sv)
print(f"The minimum k is {bsr_sse_sv.idxmin()}")
backstep_model = included[:bsr_sse_sv.idxmin()]
print(f"The selected variables are {backstep_model}")
bsr_sse_sv

The minimum k is 9
The selected variables are ['Money', 'BusEq', 'NoDur', 'Enrgy', 'Other', 'Telcm', 'Hlth ', 'Manuf', 'Shops']


1     2348.988056
2      771.982254
3      528.393322
4      368.090926
5      283.350797
6      235.289796
7      210.449663
8      208.251414
9      204.369481
10     208.891542
11     209.923549
12     209.984916
dtype: float64

### Exercise 45

Using scikit-learn to cross-validate the Backward Stepwise selected models.


#### Explanation

scikit-learn is a machine-learning library that can contains codes for many models and tasks. This includes OLS and cross-validation. Below we show how the function `cross_val_score` can be used to cross-validate a set of linear regression models.

In [14]:
# Import from sklearn
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

# Initialize a LR that will not include an intercept
lr = LinearRegression(fit_intercept=False)

# A dictionary to hold the MSE
bsr_mse_cv = {}
for i in range(1, p + 1):
    # Get the x data to use
    included_x = x[included[:i]]
    # Call cross_val_score with the model, the and and y data, and ask it so use
    # neg_mean_squared_error to cross-validate.  We will see below that
    # -t * neg_mean_squared_error is the SSE, so the maximum neg_mean_squared_error
    # will select the same model as the minimum SSE.
    # We need to take the mean since it reports the MSE for each fold of the data (5 by default)
    bsr_mse_cv[i] = cross_val_score(
        lr, included_x, y, scoring="neg_mean_squared_error"
    ).mean()

bsr_mse_cv = pd.Series(bsr_mse_cv)
# We use the maximum now
print(f"The maximum k is {bsr_mse_cv.idxmax()}")
# The variables are the same
print(f"The selected variables are {included[:bsr_mse_cv.idxmax()]}")
bsr_mse_cv

The maximum k is 9
The selected variables are ['Money', 'BusEq', 'NoDur', 'Enrgy', 'Other', 'Telcm', 'Hlth ', 'Manuf', 'Shops']


1    -5.592829
2    -1.838053
3    -1.258079
4    -0.876407
5    -0.674645
6    -0.560214
7    -0.501071
8    -0.495837
9    -0.486594
10   -0.497361
11   -0.499818
12   -0.499964
dtype: float64

#### Explanation

The negative MSE is simply 

$$ \text{Neg. MSE} = -\frac{SSE}{t} $$

so it is simple to invert back to SSE. These values are the same as above.

In [15]:
# Convert to SSE
alt_sse = -t * bsr_mse_cv
alt_sse

1     2348.988056
2      771.982254
3      528.393322
4      368.090926
5      283.350797
6      235.289796
7      210.449663
8      208.251414
9      204.369481
10     208.891542
11     209.923549
12     209.984916
dtype: float64

### Exercise 46

Repeat the cross-validation using scikit-learn using randomly selected values in each block.

#### Explanation

The default cross-validation blocks the data in the order it occurs in the inputs. If randomization is desired it is necessary to pass a cross-validation scheme that will randomize.  We do these here using `KFold` with `shuffle=True`. We also pass a `random_state` seed value to ensure that the cross-validation is reproducible. 

In [16]:
from sklearn.model_selection import KFold

lr = LinearRegression(fit_intercept=False)

bsr_mse_cv = {}
cv = KFold(5, shuffle=True, random_state=20201231)

for i in range(1, p + 1):
    bsr_mse_cv[i] = cross_val_score(
        lr, x[included[:i]], y, scoring="neg_mean_squared_error", cv=cv
    ).mean()
bsr_mse_cv = pd.Series(bsr_mse_cv)
print(f"The maximum k is {bsr_mse_cv.idxmax()}")
print(f"The selected variables are {included[:bsr_mse_cv.idxmax()]}")
bsr_mse_cv

The maximum k is 11
The selected variables are ['Money', 'BusEq', 'NoDur', 'Enrgy', 'Other', 'Telcm', 'Hlth ', 'Manuf', 'Shops', 'Utils', 'Chems']


1    -5.436108
2    -1.799260
3    -1.193152
4    -0.759860
5    -0.574028
6    -0.461058
7    -0.406622
8    -0.379140
9    -0.369990
10   -0.371283
11   -0.369444
12   -0.380203
dtype: float64

### Exercise 47

Evaluate the models selected using the sample that was _not_ used in fitting to assess the out-of-sample performance based on SSE.

#### Explanation

Finally, we can look at out-of-sample evaluation.  We simply estimate the regression coefficients in-sample and then use these with the out-of-sample x values. The residuals are the computed from these out-of-sample prediction.

In [17]:
y_oos = vwm.loc["2015":, "VWM"]
x_oos = industries["2015":]
beta = lstsq(x[bsr_model], y, rcond=None)[0]
pred = x_oos[bsr_model] @ beta
resid_oos = y_oos - x_oos[bsr_model] @ beta
oos_sse = resid_oos @ resid_oos
print(f"Best Subset OOS SSE: {oos_sse}")

Best Subset OOS SSE: 22.39851371013648


In [18]:
beta = lstsq(x[forstep_model], y, rcond=None)[0]
pred = x_oos[forstep_model] @ beta
resid_oos = y_oos - x_oos[forstep_model] @ beta
oos_sse = resid_oos @ resid_oos
print(f"Forward Stepwise OOS SSE: {oos_sse}")

Forward Stepwise OOS SSE: 22.39851371013647


In [19]:
beta = lstsq(x[backstep_model], y, rcond=None)[0]
pred = x_oos[backstep_model] @ beta
resid_oos = y_oos - x_oos[backstep_model] @ beta
oos_sse = resid_oos @ resid_oos
print(f"Backward Stepwise OOS SSE: {oos_sse}")

Backward Stepwise OOS SSE: 22.398513710136527


### Exercise 47

Compute the in-sample and out-of-sample $R^2$ for the selected models.

The out-of-sample $R^2$ defined as

$$ 1 - \frac{SSE_{OOS}}{TSS_{OOS}} $$

where the $SSE_{OOS}$ is the SSE using the predicted values and the $TSS_{OOS}$ is the TSS computed using the in-sample value (without demeaning since the models we are fitting do not include a constant). **Note**: If a model does not contain a constant, the $TSS_{OOS}$ is _not_ demeaned.

In [20]:
oos_tss = (y_oos ** 2).sum()
oos_r2 = 1 - oos_sse / oos_tss
print(f"The out-of-sample R2 for the out-of-sample period is {100*oos_r2:0.1f}%.")

The out-of-sample R2 for the out-of-sample period is 98.4%.


In [21]:
from statsmodels.api import OLS

r2 = OLS(y_oos, x_oos).fit().rsquared
print(f"The in-sample R2 for the out-of-sample period is {100*r2:0.1f}%.")

The in-sample R2 for the out-of-sample period is 99.9%.


### Exercise 48

Use scikit-learn to produce out-of-sample fits and compute the out-of-sample SSE.  Verify this value is the same as you found previously.

#### Explanation

scikit-learn is optimized for prediction and so producing out-of-sample prediction is particularly simple using the `predict` method of a fitted model. The strategy with scikit-learn models is always the same:

1. Initialize the model instance and set any configuration options.
2. Fit the model using the `fit(x, y)` method using the in-sample data
3. Predict the out-of-sample values used the `predict(x)` method using the same model instance.

In [22]:
# Specify and fit the model
lr = LinearRegression(fit_intercept=False).fit(x[bsr_model], y)
# Predictions use the coefficient estimated previously and the new data
pred = lr.predict(x_oos[bsr_model])
resid_oos = y_oos - pred
oos_sse = resid_oos @ resid_oos
oos_sse

22.39851371013648